In [1]:
import torch
import numpy as np
seed = 0
torch.manual_seed(seed)            # 为CPU设置随机种子
torch.cuda.manual_seed(seed)       # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)   # 为所有GPU设置随机种子
np.random.seed(seed)

PyTorch提供两个重要特性：
- n维张量
- 自动求导
我们将使用一个利用三次多项式来拟合y=sin(x) 的问题作为例子。这个网络有4个参数，我们通过梯度下降法来最小化网络的输出和真实输出之间的欧氏距离，以此拟合随机输入的数据。

一、张量
- 热身：Numpy
在介绍 PyTorch 之前，我们先使用 Numpy 来实现一个网络。
Numpy 提供了一个 n 维数组对象和许多操作这些数组的函数。Numpy 是一个用于科学计算的通用框架; 它不包含任何关于计算图、深度学习或梯度的内容。
但是，我们可以很容易地使用 numpy 将一个三阶多项式拟合成正弦函数，方法是通过 numpy 的一系列 operation 手动实现网络的向前和向后传递:

In [2]:
import numpy as np
import math

#创建随机的输入和输出数据
x = np.linspace(-math.pi, math.pi, 2000)  #在指定的间隔内返回均匀间隔的数字。[起始，结束，数量]
# y = np.sin(x)
y = 1 + 2 * x + 3 * x ** 2 + 4 * x ** 3

#随机生成权重
a = np.random.randn()  #randn函数返回一个或一组样本，具有标准正态分布。
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

leanring_rate = 1e-6
for t in range(10000):
    #向前传播：计算预测的 y
    #y = a + bx + cx^2 + dx^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3
    
    #计算并打印 loss
    loss = np.square(y_pred - y).sum()  #求平方和
    if t % 1000 == 999:
        print(t, loss)
        
    #反向传播来计算 a,b,c,d 关于 loss 的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()
    
    #更新权重
    a -= leanring_rate * grad_a
    b -= leanring_rate * grad_b
    c -= leanring_rate * grad_c
    d -= leanring_rate * grad_d
    
print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

999 62.42118967014553
1999 1.5067512770788012
2999 0.04011463266713382
3999 0.0011460549967731042
4999 3.42183076634501e-05
5999 1.0476789293151336e-06
6999 3.251574311288501e-08
7999 1.0163659346693077e-09
8999 3.1886119717002076e-11
9999 1.0022355737429414e-12
Result: y = 1.0000000334760732 + 1.9999999986202002 x + 2.99999999422482 x^2 + 4.000000000196264 x^3


- PyTorch：张量
Numpy 框架很棒，但不能利用GPU加速。
PyTorch 张量在概念上与 Numpy 数组类似：张量也是一个 n 维数组，PyTorch 提供了许多函数来操作这些张量。在幕后，张量可以跟踪计算图和梯度。
跟 Numpy 不同的是，PyTorch 的张量可以用GPU来加速计算，只需指定设备即可。
在这里，我们用 PyTorch 张量通过三阶多项式拟合正弦函数。同上一部分一样，我们来手动实现网络中的向前和向后传递。

In [3]:
import torch
import math


dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda")

# 创建随机输入和输出数据
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
# y = torch.sin(x)
y = 1 + 2 * x + 3 * x ** 2 + 4 * x ** 3

# 随机初始化权重
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(10000):
    # 向前传播：预测 y
    y_pred = a + b * x + c * x ** 2 + d * x **3
    
    # 计算并打印 loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 1000 == 999:
        print(t, loss)
    # 向后传播并计算 a，b，c，d关于 loss 的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()
    
    # 用梯度下降更新权重
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d
    
print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

999 117.80255126953125
1999 3.284031867980957
2999 0.09665901958942413
3999 0.0029451060108840466
4999 9.205947571899742e-05
5999 5.2841905926470645e-06
6999 2.6270663511240855e-06
7999 2.6270663511240855e-06
8999 2.6270663511240855e-06
9999 2.6270663511240855e-06
Result: y = 0.9999721050262451 + 1.999958872795105 x + 3.0000061988830566 x^2 + 4.000006198883057 x^3


二、自动求导
- PyTorch：张量和自动求导
上面的例子很简单，手动实现很容易，大型网络就不方便手动实现了。
但有了 PyTorch，我们可以使用 PyTorch 中的 autograd 包，来自动计算神经网络中的反向传递。
使用 autograd 时，网络的向前传递将定义一个计算图; 图中的节点将是张量，边将是从输入张量产生输出张量的函数。
反向传播通过这个图来计算梯度。  

听起来复杂实践起来简单。如果 x 是一个 `x.require _ grad = True` 的张量，那么x 对某个标量值的梯度则可以通过 `x.grad` 获得。  

现在我们通过 PyTorch 张量的自动求导来将三次多项式拟合正弦函数。

In [4]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# defvice = torch.device("cuda:0")

# 创建随机输入和输出数据
# 张量的 requires_grad 默认为 False
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
# y = torch.sin(x)
y = 1 + 2 * x + 3 * x ** 2 + 4 * x ** 3

# 创建四个权重张量 a, b, c, d: y = a + b x + c x^2 + d x^3
# requires_grad=True 意味着我们要计算其梯度
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # 向前传播：通过操作张量来预测 y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # 通过操作张量计算并打印 loss
    # 现在 loss 是一个形状为 （1，）的张量
    # loss.item() 用来获取 loss 的标量值
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    # 通过自动求导来计算反向传播的梯度的过程，PyTorch 会计算所有 requires_grad=True 的标量
    # 自动求导后，a.grad，b.grad等等会作为张量各自保存着 a，b 对应的关于 loss 的梯度
    loss.backward()

    # 手动更新权重（a，b，c，d）的时候，不需要自动求导的计算图追踪他们的计算过程
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # 权重更新后手动清零梯度
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None
            
print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 3416.726318359375
199 2316.1416015625
299 1571.7958984375
399 1067.831787109375
499 726.257568359375
599 494.4993896484375
699 337.08026123046875
799 230.0365447998047
899 157.1659698486328
999 107.50299072265625
1099 73.61734008789062
1199 50.47063064575195
1299 34.641231536865234
1399 23.803539276123047
1499 16.37485694885254
1599 11.277167320251465
1699 7.774991512298584
1799 5.366186618804932
1899 3.707592725753784
1999 2.5643060207366943
Result: y = 1.0438698530197144 + 1.9717165231704712 x + 2.992431402206421 x^2 + 4.00402307510376 x^3


- PyTorch：定义新的自动求导公式

实际上，每个基本的 autograd 操作符实际上是两个对 Tensors 进行操作的函数。**正向函数**从输入张量计算输出张量。**反向函数**接收输出张量相对于某个标量值的梯度，并计算输入张量相对于同一标量值的梯度。


在 PyTorch 中，我们可以通过定义 `torch.autograd.Function` 的子类来定义自己的 autograd 操作符,实现前向后向传播。然后，我们可以通过构造一个实例并像调用函数一样调用它，传递包含输入数据的 Tensors 来使用新的 autograd 操作符。

在下面的例子中，我们把模型定义为 $y=a+bP_3(c+dx)$，其中 $P_3(x)=\frac{1}{2}5x^3-3x$ 是三次勒让德多项式。
接下来自定义 autograd 函数来计算 $P_3$ 的前向和后向传播，并用其实现我们夫人模型。

In [5]:
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        """
        在向前传播当中，我们接收一个 Tensor 作为输入，返回一个 Tensor 作为输出。
        ctx 是一个用来隐藏反向传播信息的上下文对象。
        可以使用 ctx.save_for_backward 方法缓存任意对象，以便在反向传播中使用。
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)
    @staticmethod
    def backward(ctx, grad_output):
        """
        在反向传播中，我们接收一个包含关于 loss 的损失梯度的张量，我们需要计算 loss 关于 input 的梯度。
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)
    
dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")

# 创建随机输入和输出数据
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # 用 Function.apply 方法来使用我们的函数，命名为 P3
    P3 = LegendrePolynomial3.apply
    
    # 我们用自定义的自动求导操作来计算P3
    y_pred = a + b * P3(c + d * x)
    
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    loss.backward()
    
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')
    

99 209.95831298828125
199 144.66018676757812
299 100.70250701904297
399 71.03519439697266
499 50.97853088378906
599 37.40315246582031
699 28.20688247680664
799 21.973190307617188
899 17.7457275390625
999 14.877889633178711
1099 12.93176555633545
1199 11.610918045043945
1299 10.71424674987793
1399 10.105477333068848
1499 9.692105293273926
1599 9.411376953125
1699 9.220743179321289
1799 9.091285705566406
1899 9.0033597946167
1999 8.943641662597656
Result: y = -5.5117235220336624e-09 + -2.208526849746704 * P3(1.3705470092162386e-09 + 0.2554861009120941 x)


三、nn 模块

- PyTorch：nn

计算图和自动梯度是定义复杂运算符和自动求导的一个非常强大的范例，但是对于大型神经网络来说，光是自动求导还是有点不太够用。

在构建神经网络时，我们经常考虑将计算分层，某几层有可学习的参数，可以在学习过程中进行优化。

在 PyTorch 中，`nn` 包定义了一组模块，它们大致等价于神经网络层。模块的输入和输出虽然都是张量，但是也可以保存内部状态，比如包含了可学习参数的张量。`nn` 包还定义了一组常用于训练神经网络的损失函数。

在下面的例子中，我们使用 `nn` 来实现多项式网络模型：

In [6]:
import torch
import math

x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# 在这个例子中，输出：y 是（x, x^2, x^3）的线性方程，所以我们把它看作一个线性网络
# 准备张量（x, x^2, x^3）
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# 上面代码中，x.unsqueeze(-1) 的形状是（2000，1），p的形状是（3，）
# 这种情况下，广播语义可以获得一个（2000，3）尺寸的张量
# 使用 nn 包可以将我们的模型定义为一系列的层，nn.Sequential 包含其他模块，按顺序执行得到 output
# 线性模块使用线性函数计算 output ，并保存内部张量的权重和偏差。
# Flatten 层将线性层的输出压平为一维张量来匹配 y 的形状

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# nn 还包含常用损失函数的定义，在这个例子中，我们使用均方误差（MSE）作为损失函数
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):
    y_pred = model(xx)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    # 在运行向后传播之前将梯度调零
    model.zero_grad()
    
    # 反向传播: 计算模型所有可学习参数（requires_grad=True）的损失梯度
    loss.backward()
    
    # 使用梯度下降法更新权重。每个参数都是一个张量，所以我们可以像往常一样访问它的梯度
    with torch.no_grad():
        for param in model.parameters():
            param -= leanring_rate * param.grad
    
    # 你可以像访问列表的第一项一样访问模型的第一层
    linear_layer = model[0]
    # 对于线性层，其参数以“权重”和“偏置”的形式存储
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 560.7155151367188
199 374.3006286621094
299 250.87591552734375
399 169.14920043945312
499 115.02843475341797
599 79.18478393554688
699 55.44383239746094
799 39.71696853637695
899 29.297725677490234
999 22.393970489501953
1099 17.818912506103516
1199 14.786588668823242
1299 12.776483535766602
1399 11.44377613067627
1499 10.560019493103027
1599 9.973906517028809
1699 9.58509349822998
1799 9.327095031738281
1899 9.155862808227539
1999 9.04220199584961
Result: y = -0.004276792984455824 + 0.8426973819732666 x + 0.0007378174341283739 x^2 + -0.0913328230381012 x^3


- PyTorch：优化
目前为止，我们已经通过 `torch.no_grad()` 手动更新了模型的权重。对于 *像随机梯度下降* 这样的简单优化算法来说，还不算难，但是在实践中，我们经常使用更复杂的优化器来训练神经网络，比如 AdaGrad，RMSProp，Adam 等。

PyTorch 中的 `optim` 包抽象了优化算法的思想，并提供了常用优化算法的实现。

在下面的例子中，我们将像以前一样使用 nn 包来定义我们的模型，但是将使用 optim 包提供的 RMSprop 算法来优化模型:

In [7]:
import torch
import math

x = torch.linspace(-math.pi, math.pi, 2000)
# y = torch.sin(x)
y = 1 + 2 * x + 3 * x ** 2 + 4 * x ** 3

# 准备张量（x, x^2, x^3）
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# 使用 nn 来定义模型和损失函数
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# 使用 optim 定义一个优化器来为我们更新模型的权重
# 这里我们将使用 RMSprop
# optim 包含许多其他优化算法
# RMSprop 构造函数的第一个参数用来告诉优化器它应该更新哪些张量
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for t in range(2000):
    y_pred = model(xx)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    # 在向后传递之前，使用优化器对象将它要更新的变量的所有的梯度归零(这些变量是模型的可学习的权重)
    # 这是因为在默认情况下，当调用 backward（）时，梯度是在缓冲区累加的(非覆盖)
    # 查看 torch.autograd.backwards 文档可以了解更多细节
    optimizer.zero_grad()
    
    # 反向传播: 计算 loss 关于模型参数的梯度
    loss.backward()
    
    # 在优化器上调用 step() 函数来更新其参数
    optimizer.step()
    
linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 5700743.0
199 5362603.0
299 5064068.5
399 4783009.5
499 4513218.0
599 4252612.5
699 4000451.25
799 3756460.25
899 3520535.5
999 3292630.75
1099 3072722.25
1199 2860796.0
1299 2656837.0
1399 2460831.5
1499 2272766.25
1599 2092625.375
1699 1920392.25
1799 1756048.375
1899 1599573.375
1999 1450944.75
Result: y = 2.031294107437134 + 1.5065114498138428 x + 1.6419528722763062 x^2 + 1.8208980560302734 x^3


- PyTorch：自定义 nn 模块

有时候需要更复杂的模型，可以通过子类化`nn.Module`和一个`forward`来定义自己的模块。
在接下来的例子中，我们将三阶多项式作为一个自定义的 Module 子类来实现:

In [8]:
import torch
import math

class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        在构造函数中，我们实例化四个参数，并将它们作为成员参数分配
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        
    def forward(self, x):
        """
        在前向函数中，我们接受一个输入数据的张量，并且必须返回一个用来输出数据的张量。
        我们可以使用构造函数中定义的模块，也可以使用 Tensors 上的任意操作符。
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
    
    def string(self):
        """
        就像 Python 中的任何类一样，您也可以在 PyTorch 模块上自定义方法
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'
    
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 通过实例化上面定义的类来构造我们的模型
model = Polynomial3()

# 构造我们的损失函数和优化器
# SGD 构造函数中 model.parameters ()指的是 nn.Linear 模块中的可学习参数
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    y_pred = model(x)
    
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'Result: {model.string()}')

99 1369.9425048828125
199 919.3602294921875
299 618.3441162109375
399 417.12481689453125
499 282.531494140625
599 192.44390869140625
699 132.1038360595703
799 91.65908813476562
899 64.52951049804688
999 46.317256927490234
1099 34.081390380859375
1199 25.853803634643555
1299 20.31667137145996
1399 16.586793899536133
1499 14.072025299072266
1599 12.374836921691895
1699 11.228368759155273
1799 10.453095436096191
1899 9.928321838378906
1999 9.572717666625977
Result: y = -0.018849052488803864 + 0.8363862633705139 x + 0.0032517765648663044 x^2 + -0.09043512493371964 x^3


- PyTorch：控制流+权重共享

作为动态图和权重分配的一个例子，我们要实现一个非常奇怪的模型: 一个三阶-五阶多项式，每次向前传递选择一个介于3到5之间的随机数作为阶数，重复使用相同的权重计算四阶和五阶

对于这个模型，我们可以使用普通的 Python 流控制来实现循环，而且我们可以通过在定义向前传递时多次重用同一个参数来实现权重共享。

我们可以很容易地将这个模型作为一个 Module 子类来实现:

In [9]:
import random
import torch
import math


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        在构造函数中，我们实例化五个参数，并将它们分配为成员x
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))
        
    def forward(self, x):
        """
        对于模型的前向传递，我们随机选择4,5和重用参数 e 来计算这些阶的贡献。
        
        由于每次向前传递都构建一个动态计算图，因此在定义模型向前传递时，我们可以使用常规的 Python 控制流运算符，如循环或条件语句。
        
        这里我们还看到，在定义计算图时，多次重用相同的参数是非常安全的。
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y
    
    def string(self):
        """
        就像 Python 中的任何类一样，您也可以在 PyTorch 模块上自定义方法
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'
    
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 通过实例化上面定义的类来构造我们的模型
model = DynamicNet()

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

1999 218.47409057617188
3999 110.00343322753906
5999 58.60133361816406
7999 32.895172119140625
9999 21.05919647216797
11999 14.896136283874512
13999 11.889312744140625
15999 10.361847877502441
17999 9.367422103881836
19999 9.390478134155273
21999 9.0175199508667
23999 8.94487476348877
25999 8.881988525390625
27999 8.893519401550293
29999 8.617830276489258
Result: y = -0.002989751286804676 + 0.858002781867981 x + 1.8792954961099895e-06 x^2 + -0.09377000480890274 x^3 + 0.00014229427324607968 x^4 ? + 0.00014229427324607968 x^5 ?
